## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**1212. Team Scores in Football Tournament (Medium)**

**Table: Teams**

| Column Name   | Type     |
|---------------|----------|
| team_id       | int      |
| team_name     | varchar  |

team_id is the column with unique values of this table.
Each row of this table represents a single football team.
 
**Table: Matches**

| Column Name   | Type    |
|---------------|---------|
| match_id      | int     |
| host_team     | int     |
| guest_team    | int     | 
| host_goals    | int     |
| guest_goals   | int     |

match_id is the column of unique values of this table.
Each row is a record of a finished match between two different teams. 
Teams host_team and guest_team are represented by their IDs in the Teams table (team_id), and they scored host_goals and guest_goals goals, respectively.
 
You would like to compute the scores of all teams after all matches. Points are awarded as follows:
- A team receives three points if they win a match (i.e., Scored more goals than the opponent team).
- A team receives one point if they draw a match (i.e., Scored the same number of goals as the opponent team).
- A team receives no points if they lose a match (i.e., Scored fewer goals than the opponent team).

**Write a solution that selects the team_id, team_name and num_points of each team in the tournament after all described matches.**

Return the result table ordered by num_points in decreasing order. In case of a tie, order the records by team_id in increasing order.

The result format is in the following example.

**Example 1:**

**Input:** 

**Teams table:**

| team_id   | team_name    |
|-----------|--------------|
| 10        | Leetcode FC  |
| 20        | NewYork FC   |
| 30        | Atlanta FC   |
| 40        | Chicago FC   |
| 50        | Toronto FC   |

**Matches table:**
| match_id   | host_team    | guest_team    | host_goals  | guest_goals  |
|------------|--------------|---------------|-------------|--------------|
| 1          | 10           | 20            | 3           | 0            |
| 2          | 30           | 10            | 2           | 2            |
| 3          | 10           | 50            | 5           | 1            |
| 4          | 20           | 30            | 1           | 0            |
| 5          | 50           | 30            | 1           | 0            |

**Output:** 
| team_id    | team_name    | num_points    |
|------------|--------------|---------------|
| 10         | Leetcode FC  | 7             |
| 20         | NewYork FC   | 3             |
| 50         | Toronto FC   | 3             |
| 30         | Atlanta FC   | 1             |
| 40         | Chicago FC   | 0             |


In [0]:
teams_data_1212 = [
    (10, "Leetcode FC"),
    (20, "NewYork FC"),
    (30, "Atlanta FC"),
    (40, "Chicago FC"),
    (50, "Toronto FC"),
]

teams_columns_1212 = ["team_id", "team_name"]
teams_df_1212 = spark.createDataFrame(teams_data_1212, teams_columns_1212)
teams_df_1212.show()

matches_data_1212 = [
    (1, 10, 20, 3, 0),
    (2, 30, 10, 2, 2),
    (3, 10, 50, 5, 1),
    (4, 20, 30, 1, 0),
    (5, 50, 30, 1, 0),
]

matches_columns_1212 = ["match_id", "host_team", "guest_team", "host_goals", "guest_goals"]
matches_df_1212 = spark.createDataFrame(matches_data_1212, matches_columns_1212)
matches_df_1212.show()

In [0]:
host_df_1212 = matches_df_1212.select(
    col("host_team").alias("team_id"),
    when(col("host_goals") > col("guest_goals"), 3)
    .when(col("host_goals") == col("guest_goals"), 1)
    .otherwise(0)
    .alias("points")
)

In [0]:
guest_df_1212 = matches_df_1212.select(
    col("guest_team").alias("team_id"),
    when(col("guest_goals") > col("host_goals"), 3)
    .when(col("guest_goals") == col("host_goals"), 1)
    .otherwise(0)
    .alias("points")
)

In [0]:
points_df_1212 = host_df_1212.union(guest_df_1212)

In [0]:
team_points_df_1212 = points_df_1212.groupBy("team_id").agg(sum("points").alias("num_points"))

In [0]:
team_points_df_1212\
    .join(teams_df_1212, on="team_id", how="right").fillna(0)\
        .select("team_id", "team_name", "num_points") \
            .orderBy(col("num_points").desc(), col("team_id")).show()